In [1]:
import pandas as pd
import numpy as np
import ipdb


all_videos_csv = "../assets/training_data_20250710_103210_ffprobe_metadata_vfr.csv"
val_participants_txt = "../assets/val_split.txt"

In [2]:
all_videos_df = pd.read_csv(all_videos_csv)
val_participants = set(pd.read_csv(val_participants_txt, header=None, names=["participant"])["participant"])

In [3]:
available_videos_df = all_videos_df[
    all_videos_df["participant_id"].isin(val_participants) &
    all_videos_df["modifiers"].isin({"raw", "facefusion-v1", "facefusion-identity-swap-v1"}) &
    (all_videos_df["resolution"] == "1280x720") &
    (all_videos_df["video_codec_original"] == "h264")
]

In [4]:
# Keep only those rows where we have a real and a fake version
candidate_item_ids = set(available_videos_df[available_videos_df["modifiers"] == "raw"]["item_id"])
available_videos_df = available_videos_df[available_videos_df["item_id"].isin(candidate_item_ids)]

def select_pair(g: pd.DataFrame) -> pd.DataFrame:
    if len(g) < 2:
        # invalid group
        return None
        
    real_mask = g["modifiers"] == "raw"
    assert real_mask.sum() == 1
    
    real_row_idx = g[real_mask].index[0]
    fake_row_idx = g[~real_mask].sample(1, random_state=g.loc[real_row_idx, "participant_id"]).index[0]

    return g.loc[[real_row_idx, fake_row_idx]]

selected_videos_df = available_videos_df.groupby("item_id", as_index=False, sort=False).apply(
    lambda g: select_pair(g), include_groups=True).reset_index(drop=True)

assert np.all(selected_videos_df.value_counts("item_id") == 2)

# Select a subset of the dataframe
rng = np.random.default_rng(2000)
selected_item_ids = rng.choice(selected_videos_df["item_id"].unique(), size=200, replace=False)

selected_videos_df = selected_videos_df[selected_videos_df["item_id"].isin(selected_item_ids)]

print(f"Duration: {selected_videos_df["duration"].sum() / 60 / 60} hours")

Duration: 5.054677292777779 hours


/tmp/ipykernel_88330/3428782996.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  selected_videos_df = available_videos_df.groupby("item_id", as_index=False, sort=False).apply(


In [5]:
selected_videos_df[["resolution", "video_codec_original", "pix_fmt", "modifiers"]].value_counts()

resolution  video_codec_original  pix_fmt   modifiers                  
1280x720    h264                  yuvj420p  raw                            150
                                  yuv420p   facefusion-v1                  140
                                            facefusion-identity-swap-v1     60
                                            raw                             50
Name: count, dtype: int64

In [6]:
selected_videos_df.to_csv("../assets/2025_07_16-selected_videos.csv", index=False)